In [ ]:
!pip install seaborn
!pip install nltk
!pip install wordcloud

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

In [94]:
import os.path as path
from pyspark.sql import SparkSession
from pyspark.sql import column
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
from pyspark.ml.feature import StringIndexer, VectorAssembler,OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier

In [95]:
def init_spark():
  return SparkSession \
        .builder \
        .appName("Big data project") \
        .config("spark.executor.memory", "8g") \
        .config("spark.driver.memory", "8g") \
        .getOrCreate()

spark = init_spark()

In [96]:
def get_data_dataframe(dir="final_dataset.csv"):
    # Specify the directory where the CSV files are saved
    csv_directory = dir
    
    # Read the CSV files back into a DataFrame
    final_df_read = spark.read.option("header", "true").csv(csv_directory)
    
    # Add a random column to shuffle data randomly
    shuffled_df = final_df_read.withColumn("rand", rand())
    shuffled_df = shuffled_df.orderBy("rand")
    final_df_read = shuffled_df.drop("rand")
    return final_df_read



In [97]:
df=get_data_dataframe("final_dataset4_imdb.csv").limit(50000)
df = df.filter(col("genre") != "thriller")

In [98]:
df.count()

41708

In [99]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stopword = set(stopwords.words('english'))

def preprocessing(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation using regular expressions
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove specific characters #, @, and $
    text = re.sub(r'[#@\$]', '', text)
    
    # tokenize and convert to list
    tokens = word_tokenize(text)
    
    ## Lemmatize it 
    lemmatizer  = WordNetLemmatizer()
    
    ## lemmatize each token
   # text = [lemmatizer.lemmatize(token) for token in tokens]
    text = text.split()
    
    text = [word for word in text if word not in stopword]
    
    
    return " ".join(text) 

[nltk_data] Downloading package punkt to /home/umang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/umang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/umang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [100]:
# pdd=df.toPandas()
# # pdd.drop_duplicates(inplace = True)
# # pdd.dropna( inplace = True )
# pdd.size

In [101]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

In [102]:
# df=combined_train_df

In [103]:
# df
# 

In [104]:
# Remove duplicate rows
df = df.dropDuplicates()

# Remove rows with null values
# df_no_null = df_no_duplicates.dropna()


In [105]:
df.count()

41708

In [106]:
# df=df_no_duplicates

In [107]:
# df = df.select([sum (col(c).isNull().cast("int")).alias(c) for c in df.columns])

# # Show the count of null values for each column
# df.show()


In [108]:
df = df.dropna()

In [109]:
# df = df.select([sum (col(c).isNull().cast("int")).alias(c) for c in df.columns])

# # Show the count of null values for each column
# df.show()
# Tokenization, stop word removal, and TF-IDF transformation for text data
tokenizer = RegexTokenizer(inputCol="description", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="description_clean")

# Encoding genre labels
genre_indexer = StringIndexer(inputCol="genre", outputCol="label")


pipeline = Pipeline(stages=[tokenizer,remover,vectorizer, idf])
transformed_train_df = pipeline.fit(df).transform(df)
# Combine the preprocessed text features with the genre labels
combined_train_df = genre_indexer.fit(transformed_train_df).transform(transformed_train_df)


In [110]:
df=combined_train_df

In [111]:
# Split the data into training and testing sets
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Show the number of rows in each set
print("Training set count:", train_df.count())
print("Testing set count:", test_df.count())

24/03/28 21:53:26 WARN DAGScheduler: Broadcasting large task binary with size 1102.4 KiB
24/03/28 21:53:28 WARN DAGScheduler: Broadcasting large task binary with size 1102.4 KiB


Training set count: 33362


Testing set count: 8342


In [112]:
df

DataFrame[movie_id: string, movie_name: string, genre: string, description: string, words: array<string>, filtered_words: array<string>, raw_features: vector, description_clean: vector, label: double]

In [113]:
# Train the model
# model = pipeline.fit(train_df)
# Define the Random Forest classifier
rf_classifier = RandomForestClassifier(labelCol="label", featuresCol="description_clean", numTrees=50)

# Train the Random Forest classifier
model = rf_classifier.fit(df)

# Make predictions
# predictions = model.transform(test_df)

24/03/28 21:53:30 WARN DAGScheduler: Broadcasting large task binary with size 1083.6 KiB
24/03/28 21:53:30 WARN DAGScheduler: Broadcasting large task binary with size 1083.7 KiB
24/03/28 21:53:31 WARN DAGScheduler: Broadcasting large task binary with size 1483.6 KiB
24/03/28 21:53:41 WARN DAGScheduler: Broadcasting large task binary with size 1903.7 KiB
24/03/28 21:53:50 WARN MemoryStore: Not enough space to cache rdd_873_0 in memory! (computed 4.0 GiB so far)
24/03/28 21:53:50 WARN BlockManager: Persisting block rdd_873_0 to disk instead.
24/03/28 21:54:04 WARN MemoryStore: Not enough space to cache rdd_873_0 in memory! (computed 4.0 GiB so far)
24/03/28 21:54:08 WARN DAGScheduler: Broadcasting large task binary with size 1961.5 KiB
24/03/28 21:54:11 WARN MemoryStore: Not enough space to cache rdd_873_0 in memory! (computed 4.0 GiB so far)
24/03/28 21:54:15 WARN DAGScheduler: Broadcasting large task binary with size 2041.5 KiB
24/03/28 21:54:18 WARN MemoryStore: Not enough space to ca

In [114]:
predictions = model.transform(test_df)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

24/03/28 21:54:37 WARN DAGScheduler: Broadcasting large task binary with size 1364.5 KiB


Accuracy: 0.4381443298969072


In [115]:
predictions.select("genre","label","prediction").head(30)

24/03/28 21:54:40 WARN DAGScheduler: Broadcasting large task binary with size 1353.4 KiB


[Row(genre='crime', label=0.0, prediction=1.0),
 Row(genre='adventure', label=2.0, prediction=1.0),
 Row(genre='horror', label=3.0, prediction=2.0),
 Row(genre='action', label=4.0, prediction=1.0),
 Row(genre='adventure', label=2.0, prediction=1.0),
 Row(genre='adventure', label=2.0, prediction=4.0),
 Row(genre='adventure', label=2.0, prediction=1.0),
 Row(genre='adventure', label=2.0, prediction=0.0),
 Row(genre='horror', label=3.0, prediction=2.0),
 Row(genre='horror', label=3.0, prediction=1.0),
 Row(genre='action', label=4.0, prediction=2.0),
 Row(genre='adventure', label=2.0, prediction=1.0),
 Row(genre='crime', label=0.0, prediction=0.0),
 Row(genre='adventure', label=2.0, prediction=1.0),
 Row(genre='adventure', label=2.0, prediction=2.0),
 Row(genre='horror', label=3.0, prediction=1.0),
 Row(genre='adventure', label=2.0, prediction=1.0),
 Row(genre='action', label=4.0, prediction=2.0),
 Row(genre='romance', label=1.0, prediction=2.0),
 Row(genre='adventure', label=2.0, predicti

In [116]:


def print_statistis(predictions):

    # Compute raw scores on the test set
    predictionAndLabels = predictions.rdd.map(lambda lp: (lp.prediction, lp.label))
    
    # Instantiate metrics object
    metrics = MulticlassMetrics(predictionAndLabels)
    confusion_matrix = metrics.confusionMatrix().toArray()
    
    # Overall statistics
    precision = metrics.precision(1.0)
    recall = metrics.recall(1.0)
    f1Score = metrics.fMeasure(1.0)
    total_predictions = confusion_matrix.sum(axis=1)
    print("Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)
    
    # Statistics by class
    labels = predictions.rdd.map(lambda lp: lp.label).distinct().collect()
    accuracies = {}
    for label in sorted(labels):
        # print("______________________"+mapping[label]+"_____________________")
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))
        
    
    # Calculate the accuracy for each label
    print(total_predictions)
    for label in range(len(total_predictions)):
        correct_predictions = confusion_matrix[label, label]
        accuracy = correct_predictions / total_predictions[label]
        accuracies[label] = accuracy
    
    # Print accuracies for each label
    for label, accuracy in accuracies.items():
        print("Accuracy for label %s: %s" % (label, accuracy))


In [117]:
print_statistis(predictions)

24/03/28 21:54:43 WARN DAGScheduler: Broadcasting large task binary with size 1380.4 KiB
24/03/28 21:54:46 WARN DAGScheduler: Broadcasting large task binary with size 1390.5 KiB
24/03/28 21:54:49 WARN DAGScheduler: Broadcasting large task binary with size 1383.1 KiB


Summary Stats
Precision = 0.3878048780487805
Recall = 0.7522176227084565
F1 Score = 0.5117682558841279


Class 0.0 precision = 0.4775742261528743
Class 0.0 recall = 0.45652173913043476
Class 0.0 F1 Measure = 0.46681074405680767
Class 1.0 precision = 0.3878048780487805
Class 1.0 recall = 0.7522176227084565
Class 1.0 F1 Measure = 0.5117682558841279
Class 2.0 precision = 0.4024787997390737
Class 2.0 recall = 0.3681384248210024
Class 2.0 F1 Measure = 0.3845434714864444
Class 3.0 precision = 0.5690737833594977
Class 3.0 recall = 0.44505831798649476
Class 3.0 F1 Measure = 0.4994832931450223
Class 4.0 precision = 0.42410714285714285
Class 4.0 recall = 0.16863905325443787
Class 4.0 F1 Measure = 0.24132091447925486
[1656. 1691. 1676. 1629. 1690.]
Accuracy for label 0: 0.45652173913043476
Accuracy for label 1: 0.7522176227084565
Accuracy for label 2: 0.3681384248210024
Accuracy for label 3: 0.44505831798649476
Accuracy for label 4: 0.16863905325443787
